In [33]:
!hdfs dfs -ls /

Found 4 items
drwx------   - mapred hadoop          0 2025-04-15 18:32 /hadoop
drwxrwxrwt   - hdfs   hadoop          0 2025-04-15 18:32 /tmp
drwxrwxrwt   - hdfs   hadoop          0 2025-04-16 17:52 /user
drwxrwxrwt   - hdfs   hadoop          0 2025-04-15 18:32 /var


In [36]:
ls

bible.txt  HW2.ipynb  mapper.py  reducer.py  Untitled1.ipynb
hello.txt  map2.py    red2.py    sites.csv


In [35]:
!hdfs dfs -ls /user/files_for_hw

Found 2 items
-rw-r--r--   1 ubuntu hadoop    4047392 2025-04-16 18:01 /user/files_for_hw/bible.txt
-rw-r--r--   1 ubuntu hadoop   36443383 2025-04-16 18:05 /user/files_for_hw/sites.csv


In [36]:
ls


bible.txt  hello.txt  HW2.ipynb  sites.csv  Untitled1.ipynb


In [37]:
! hdfs dfs -ls /user


Found 3 items
drwxr-xr-x   - ubuntu hadoop          0 2025-04-16 18:05 /user/files_for_hw
drwxr-xr-x   - hive   hadoop          0 2025-04-15 18:32 /user/hive
drwxr-xr-x   - ubuntu hadoop          0 2025-04-15 18:35 /user/ubuntu


In [38]:
!hadoop fs -put bible.txt /user/files_for_hw

put: `/user/files_for_hw/bible.txt': File exists


In [65]:
!hadoop fs -put sites.csv /user/files_for_hw

put: `/user/files_for_hw/sites.csv': File exists


In [40]:
!hdfs dfs -ls /

Found 4 items
drwx------   - mapred hadoop          0 2025-04-15 18:32 /hadoop
drwxrwxrwt   - hdfs   hadoop          0 2025-04-15 18:32 /tmp
drwxrwxrwt   - hdfs   hadoop          0 2025-04-16 17:52 /user
drwxrwxrwt   - hdfs   hadoop          0 2025-04-15 18:32 /var


In [41]:
%%bash


hdfs dfs -ls /

Found 4 items
drwx------   - mapred hadoop          0 2025-04-15 18:32 /hadoop
drwxrwxrwt   - hdfs   hadoop          0 2025-04-15 18:32 /tmp
drwxrwxrwt   - hdfs   hadoop          0 2025-04-16 17:52 /user
drwxrwxrwt   - hdfs   hadoop          0 2025-04-15 18:32 /var


In [13]:
%%writefile mapper.py

import sys

def mapper():
    for line in sys.stdin:
        line = line.strip()
        if line:
            words = line.split()
            for word in words:
                if len(word) > 4:
                    print(f"{word}\t 1")

if __name__ == "__main__":
    mapper()

Overwriting mapper.py


In [14]:
%%writefile reducer.py
import sys

def reducer():
    current_word = None
    current_count = 0
    for line in sys.stdin:
        word, count = line.strip().split('\t')
        count = int(count)
        
        if word == current_word:
            current_count += count
        else:
            if current_word is not None:
                print(f"{current_word}\t{current_count}")
            current_word = word
            current_count = count
    
    if current_word is not None:
        print(f"{current_word}\t{current_count}")

if __name__ == "__main__":
    reducer()

Overwriting reducer.py


In [16]:
%%bash
## пример запуска скриптов на неймноде для проверки их работы
cat bible.txt | python3 mapper.py | sort -k1,1 | python3 reducer.py | sort -k2,2nr | head -n 50

shall	9658
which	4119
their	3801
there	1799
before	1701
against	1587
shalt	1586
children	1559
said,	1556
them,	1549
LORD,	1308
saying,	1225
house	1221
people	1194
thee,	1170
every	1150
saith	1135
after	1053
Israel	1025
because	985
these	921
among	894
thine	863
great	835
brought	814
things	780
Jesus	775
neither	762
should	757
according	725
Israel,	723
forth	720
David	687
them.	674
bring	668
therefore	662
LORD.	587
behold,	572
heard	559
called	556
people,	512
spake	508
Behold,	501
hundred	501
thee.	486
Moses	481
given	476
heart	461
those	436
about	431


In [9]:
!hdfs dfs -mkdir /user/files_for_hw/word_count_task

mkdir: `/user/files_for_hw/word_count_task': File exists


In [64]:
!hdfs dfs -mkdir /user/files_for_hw/url_rank_task

In [66]:
!hdfs dfs -ls /user/files_for_hw

Found 4 items
-rw-r--r--   1 ubuntu hadoop    4047392 2025-04-16 18:01 /user/files_for_hw/bible.txt
-rw-r--r--   1 ubuntu hadoop   36443383 2025-04-16 18:05 /user/files_for_hw/sites.csv
drwxr-xr-x   - ubuntu hadoop          0 2025-04-18 20:58 /user/files_for_hw/url_rank_task
drwxr-xr-x   - ubuntu hadoop          0 2025-04-18 19:50 /user/files_for_hw/word_count_task


In [17]:
%%bash
## шаблон для запуска MR таски

# обязательная чистка директории, куда будем складывать результат отрабоки mr
hdfs dfs -rm -r /user/files_for_hw/word_count_task || true

# запус mr таски с указанием пути до нужного jar
hadoop jar /usr/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapreduce.job.name="word-count" \
    -files ./mapper.py,./reducer.py \
    -mapper "python3 mapper.py" \
    -reducer "python3 reducer.py" \
    -input /user/files_for_hw/bible.txt \
    -output /user/files_for_hw/word_count_task

Deleted /user/files_for_hw/word_count_task
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-3.2.2.jar] /tmp/streamjob4291457022299290327.jar tmpDir=null


2025-04-18 19:49:28,689 INFO client.RMProxy: Connecting to ResourceManager at rc1b-dataproc-m-ifco7qks15ipul9r.mdb.yandexcloud.net/10.129.0.28:8032
2025-04-18 19:49:28,911 INFO client.AHSProxy: Connecting to Application History server at rc1b-dataproc-m-ifco7qks15ipul9r.mdb.yandexcloud.net/10.129.0.28:10200
2025-04-18 19:49:28,947 INFO client.RMProxy: Connecting to ResourceManager at rc1b-dataproc-m-ifco7qks15ipul9r.mdb.yandexcloud.net/10.129.0.28:8032
2025-04-18 19:49:28,948 INFO client.AHSProxy: Connecting to Application History server at rc1b-dataproc-m-ifco7qks15ipul9r.mdb.yandexcloud.net/10.129.0.28:10200
2025-04-18 19:49:29,213 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/ubuntu/.staging/job_1745004030788_0007
2025-04-18 19:49:30,396 INFO mapred.FileInputFormat: Total input files to process : 1
2025-04-18 19:49:30,544 INFO mapreduce.JobSubmitter: number of splits:30
2025-04-18 19:49:30,724 INFO mapreduce.JobSubmitter: Submitting 

In [18]:
%%bash
## запустите эту команду, чтобы вывести счетчик определенных слов, которые мы указали на grep
## Это нам будет необходимо для визуального анализа результата работы вашего скрипта
## в sort можете указать тот разделитель колонок, с которым у вас результат выплевывает редьюсер

hdfs dfs -cat /user/files_for_hw/word_count_task/* | grep  -E 'lord\.|god\.|pray\.' | sort -t$'\t' -k2.2nr  | head -n 3

lord.	17
pray.	5


In [60]:
%%writefile map2.py

import sys

def mapper():
    for line in sys.stdin:
        
        
        if not line:
            continue
        
        
        url, time = line.strip().split(';')
        date = time.split(' ')[0]
        print(f"{date}\t{url}\t1")

        
if __name__ == "__main__":
    mapper()

Overwriting map2.py


In [58]:
%%writefile red2.py
import sys

def reducer():
    result_count = 0
    
    result_date = None
    
    result_url = None
    

    for line in sys.stdin:
        
        if not line:
            continue
        
        col = line.strip().split('\t')
        
        if len(col) != 3:
            continue
        
        if col[0] == result_date and col[1] == result_url:
            result_count += int(col[2])
        
        else:
            if result_date:
                print(f"{result_date}\t{result_url}\t{result_count}")
            
            result_date = col[0]
            result_url = col[1]
            result_count = int(col[2])

    # Выводим последнюю запись
    if result_date:
        print(f"{result_date}\t{result_url}\t{result_count}")

if __name__ == "__main__":
    reducer()              
        

Overwriting red2.py


In [62]:
%%bash

cat sites.csv | python3 map2.py | sort -k1,1 | python3 red2.py | sort -k1,1 -k3,3nr | awk '$1 != prev {prev=$1; cnt=1} cnt <=5 {print; cnt++}'

2024-05-26	https://gonzales-bautista.com/	335
2024-05-26	http://smith.com/	235
2024-05-26	https://www.smith.com/	221
2024-05-26	https://smith.com/	212
2024-05-26	http://www.smith.com/	212
2024-05-27	https://gonzales-bautista.com/	376
2024-05-27	https://www.smith.com/	270
2024-05-27	https://smith.com/	236
2024-05-27	http://smith.com/	215
2024-05-27	http://www.smith.com/	208
2024-05-28	https://gonzales-bautista.com/	368
2024-05-28	https://smith.com/	256
2024-05-28	https://www.smith.com/	251
2024-05-28	http://smith.com/	224
2024-05-28	http://www.smith.com/	204
2024-05-29	https://gonzales-bautista.com/	402
2024-05-29	https://www.smith.com/	242
2024-05-29	http://www.smith.com/	223
2024-05-29	https://smith.com/	220
2024-05-29	http://smith.com/	206
2024-05-30	https://gonzales-bautista.com/	353
2024-05-30	https://smith.com/	246
2024-05-30	https://www.smith.com/	239
2024-05-30	http://smith.com/	229
2024-05-30	http://www.smith.com/	225
2024-05-31	https://gonzales-bautista.com/	374
2024-05-31	htt

In [73]:
%%bash
## шаблон для запуска MR таски

# обязательная чистка директории, куда будем складывать результат отрабоки mr
hdfs dfs -rm -r /user/files_for_hw/url_rank_task || true

# запус mr таски с указанием пути до нужного jar
hadoop jar /usr/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapreduce.job.name="url-rank" \
    -D mapreduce.job.reduces=1 \
    -files ./map2.py,./red2.py \
    -mapper "python3 map2.py" \
    -reducer "python3 red2.py" \
    -input /user/files_for_hw/sites.csv \
    -output /user/files_for_hw/url_rank_task

Deleted /user/files_for_hw/url_rank_task
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-3.2.2.jar] /tmp/streamjob5887909444261507697.jar tmpDir=null


2025-04-18 21:08:11,968 INFO client.RMProxy: Connecting to ResourceManager at rc1b-dataproc-m-ifco7qks15ipul9r.mdb.yandexcloud.net/10.129.0.28:8032
2025-04-18 21:08:12,195 INFO client.AHSProxy: Connecting to Application History server at rc1b-dataproc-m-ifco7qks15ipul9r.mdb.yandexcloud.net/10.129.0.28:10200
2025-04-18 21:08:12,237 INFO client.RMProxy: Connecting to ResourceManager at rc1b-dataproc-m-ifco7qks15ipul9r.mdb.yandexcloud.net/10.129.0.28:8032
2025-04-18 21:08:12,238 INFO client.AHSProxy: Connecting to Application History server at rc1b-dataproc-m-ifco7qks15ipul9r.mdb.yandexcloud.net/10.129.0.28:10200
2025-04-18 21:08:12,490 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/ubuntu/.staging/job_1745004030788_0011
2025-04-18 21:08:13,233 INFO mapred.FileInputFormat: Total input files to process : 1
2025-04-18 21:08:13,319 INFO mapreduce.JobSubmitter: number of splits:30
2025-04-18 21:08:13,496 INFO mapreduce.JobSubmitter: Submitting 

In [75]:
%%bash
## запустите эту команду, чтобы вывести результат работы по определенным компаниям, которые мы указали на grep
## Это нам будет необходимо для визуального анализа результата работы вашего скрипта
## в sort можете указать тот разделитель колонок, с которым у вас результат выплевывает редьюсер
## укажите путь до той директории на hdfs, куда вы складывали результат

hdfs dfs -cat /user/files_for_hw/url_rank_task/* | grep -E '2024-05-28|2024-06-02|2024-05-30' | column -t -s$'\t' 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

